<h1>Wirecell Clustering v0_30_2</h1>

<h6>
<u>File</u>: Clustering v0_30_2.ipynb <br>
<br>
<u>Author</u>: Ewerton Belchior <br>
<br>
<u>Goal</u>: General information on how to setup sbndcode and wirecell for clustering evaluation. <br>
<br>

<h4>0) Create your working area

In [ ]:
export myworkdir=/exp/sbnd/app/users/ebatista/wirecell-clustering/work
mkdir -p $myworkdir

# go to your working area
cd $myworkdir

<h4>1) Setup sbndcode

In [ ]:
/cvmfs/oasis.opensciencegrid.org/mis/apptainer/current/bin/apptainer shell --shell=/bin/bash -B /cvmfs,/exp,/nashome,/pnfs/sbn,/pnfs/sbnd,/opt,/run/user,/etc/hostname,/etc/hosts,/etc/krb5.conf --ipc --pid /cvmfs/singularity.opensciencegrid.org/fermilab/fnal-dev-sl7:latest

In [ ]:
source /cvmfs/sbnd.opensciencegrid.org/products/sbnd/setup_sbnd.sh
setup sbndcode v10_04_07 -q e26:prof

<h4>2) Build WCT<br><br>
&nbsp;&nbsp;&nbsp;&nbsp;- This step should be done *only* if you are debugging any wct module locally (skip it otherwise)

In [ ]:
# go to your working area
cd $myworkdir

git clone -b 0.30.2 https://github.com/WireCell/wire-cell-toolkit.git wire-cell-toolkit_v0.30.2

cd wire-cell-toolkit_v0.30.2
mkdir -p install
    
./wcb configure \
--prefix=$myworkdir/wire-cell-toolkit_v0.30.2/install \
--with-spdlog=/cvmfs/larsoft.opensciencegrid.org/products/spdlog/v1_9_2 \
--with-spdlog-include=/cvmfs/larsoft.opensciencegrid.org/products/spdlog/v1_9_2/Linux64bit+3.10-2.17-e26-prof/include \
--with-spdlog-lib=/cvmfs/larsoft.opensciencegrid.org/products/spdlog/v1_9_2/Linux64bit+3.10-2.17-e26-prof/lib64 \
--with-jsoncpp=/cvmfs/larsoft.opensciencegrid.org/products/jsoncpp/v1_9_5a \
--with-jsoncpp-include=/cvmfs/larsoft.opensciencegrid.org/products/jsoncpp/v1_9_5a/Linux64bit+3.10-2.17-e26-prof/include \
--with-jsoncpp-lib=/cvmfs/larsoft.opensciencegrid.org/products/jsoncpp/v1_9_5a/Linux64bit+3.10-2.17-e26-prof/lib \
--with-eigen=/cvmfs/larsoft.opensciencegrid.org/products/eigen/v23_08_01_66e8f \
--with-eigen-include=/cvmfs/larsoft.opensciencegrid.org/products/eigen/v23_08_01_66e8f/include/eigen3 \
--with-jsonnet-lib=/cvmfs/larsoft.opensciencegrid.org/products/jsonnet/v0_17_0a/Linux64bit+3.10-2.17-e20-prof/lib \
--with-jsonnet-include=/cvmfs/larsoft.opensciencegrid.org/products/jsonnet/v0_17_0a/Linux64bit+3.10-2.17-e20-prof/include \
--boost-includes=/cvmfs/larsoft.opensciencegrid.org/products/boost/v1_82_0/Linux64bit+3.10-2.17-e26-prof/include \
--boost-libs=/cvmfs/larsoft.opensciencegrid.org/products/boost/v1_82_0/Linux64bit+3.10-2.17-e26-prof/lib
./wcb -p -v --notests install

# go back to your working area
cd $myworkdir

In [ ]:
export LD_LIBRARY_PATH=$myworkdir/wire-cell-toolkit_v0.30.2/install/lib:$LD_LIBRARY_PATH

<h4>3) Build larreco <br><br>
&nbsp;&nbsp;&nbsp;&nbsp;-This step is needed for using a modified CellTree_Module.cc with truthDepos clustering (clone/build just once)

In [ ]:
# go to your working area
cd $myworkdir

mkdir larsoft_v10_04_07
cd larsoft_v10_04_07
mrb newDev
source localProducts_larsoft_v10_04_07_e26_prof/setup
cd srcs
git clone https://github.com/ebelchio12/larreco.git
cd larreco
git fetch --all
git checkout ebelchio_edepos_clustering
cd ..
mrb uc
cd $MRB_BUILDDIR
mrbsetenv
mrb i --generator ninja
mrbslp
cd -

<h5>Setup larreco everytime you log in on gpvm

In [ ]:
# Use local larreco only to cluster thruthDepos based on their timming, using CellTree_module. If not, no need to setup local products.
source $myworkdir/larsoft_v10_04_07/localProducts_larsoft_v10_04_07_e26_prof/setup
mrbslp

In [ ]:
# example running
lar -n1 -c celltree_sbnd_ew.fcl /exp/sbnd/data/users/ebatista/wirecell-clustering/out/enuelastic/g4.root

<h4>4) Setup clustering

In [ ]:
# go to your working area
cd $myworkdir
mkdir img-clus-configs

# copy wct configurations and data to your working directory
cp -r /exp/sbnd/app/users/ebatista/wirecell-clustering/work/wire-cell-toolkit_v0.30.2/cfg img-clus-configs/
cp -r /exp/sbnd/app/users/ebatista/wirecell-clustering/work/wire-cell-data img-clus-configs/
cp -r /exp/sbnd/app/users/ebatista/wirecell-clustering/work/wcp-porting-img .

In [ ]:
export WIRECELL_PATH=$myworkdir/img-clus-configs/cfg:$WIRECELL_PATH
#export WIRECELL_PATH=/exp/sbnd/app/users/yuhw/wire-cell-data:$WIRECELL_PATH
export WIRECELL_PATH=$myworkdir/img-clus-configs/wire-cell-data:$WIRECELL_PATH

In [ ]:
# example running
lar -n1 --nskip 10 -c wcp-porting-img/sbnd/wcls-img-clus-data.fcl /exp/sbnd/data/users/ebatista/wirecell-clustering/eval/run18005/sigproc.root
python wcp-porting-img/sbnd/merge-zip.py
source wcp-porting-img/upload-to-bee.sh mabc.zip

<h4>5) Run the workflow for sample generation, signal processing, imaging and clustering, then save output in txt format</h4>

([Click here to open create_cluster_samples.sh](http://localhost:8888/notebooks/OneDrive/Documentos/backup-2025-xx-xx/Meetings/WireCell/Clustering/wirecell-clustering/create_cluster_samples.sh?))

In [ ]:
cd /exp/sbnd/app/users/ebatista/wirecell-clustering/work
bash create_cluster_samples.sh